# Econ 148 Project 3

#### Warming increases the risk of civil war in Africa

In [13]:
!pip install stargazer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from stargazer.stargazer import Stargazer

In [14]:
df_gseasSSA = pd.read_csv("gseasSSA.csv")
df_mss_countries_new = pd.read_csv("mss_countries_new.csv")
df_precip_climatology = pd.read_csv("precip_climatology.csv")
df_croparea_weight = pd.read_csv("croparea_weight.csv", encoding="windows-1252")
df_clim_conflict = pd.read_csv("clim_conflict_for_R.csv")

In [15]:
df_gseasSSA.head()

,code,country,beg_month,end_month
0,8,Angola,-2,3
1,17,Botswana,-1,4
2,26,Benin,3,8
3,34,Burundi,-3,2
4,37,Chad,5,10


In [16]:
df_mss_countries_new.head()

,country,year
0,Guinea-Bissau,1981
1,Guinea-Bissau,1982
2,Guinea-Bissau,1983
3,Guinea-Bissau,1984
4,Guinea-Bissau,1985


In [17]:
df_precip_climatology.head()

,Unnamed: 0,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,sahel,1.709537,2.492318,5.714172,16.130378,41.333336,76.099117,139.665724,164.860457,98.040216,36.330694,6.243360,2.471161
1,west_afr,8.209585,21.885972,48.073384,81.189124,128.830185,187.294634,211.482483,235.918114,212.480851,126.953568,40.818007,14.821853
2,cent_afr,150.822840,135.684500,172.917389,131.626008,66.964828,38.162833,41.202987,59.097322,81.299915,117.013004,160.153958,166.075571
3,east_afr,78.856493,77.336063,117.425652,139.091246,94.367546,57.119627,79.829072,81.796797,64.329445,63.728370,86.672489,84.657042
4,south_afr,173.127541,159.389011,126.137167,58.982098,20.821174,13.856495,13.782481,12.926650,15.509177,40.318556,91.751808,159.689238


In [18]:
df_croparea_weight.head()

,countries,item,element,2000,region
0,Angola,"Cereals,Total +",Area Harvested (Ha),890677,5.0
1,Benin,"Cereals,Total +",Area Harvested (Ha),901324,2.0
2,Botswana,"Cereals,Total +",Area Harvested (Ha),189542,5.0
3,Burkina Faso,"Cereals,Total +",Area Harvested (Ha),2661349,2.0
4,Burundi,"Cereals,Total +",Area Harvested (Ha),196300,4.0


In [19]:
df_clim_conflict.head()

,country,countryisocode,year_actual,ccode,war_prio,any_prio,war_prio_on,any_prio_on,gdp,gdp_l,...,cru_prec_diftrend,cru_prec_diftrend_lag,gdpcapgrowth,gdpcapgrowth_lag,gdpcapgrowth_lag2,gdpcap1980,polity2,polity2_lag,polity2_lag2,polity1980
0,Guinea-Bissau,GNB,1981,404,0,0,0.0,0.0,647.1996,521.2217,...,0.261924,-0.399240,24.169734,NaN,NaN,521.2217,-7,-7,-7.0,-7
1,Guinea-Bissau,GNB,1982,404,0,0,0.0,0.0,586.6083,647.1996,...,0.070554,0.261924,-9.362073,24.169734,NaN,521.2217,-7,-7,-7.0,-7
2,Guinea-Bissau,GNB,1983,404,0,0,0.0,0.0,636.3018,586.6083,...,-0.371771,0.070554,8.471326,-9.362073,24.169734,521.2217,-7,-7,-7.0,-7
3,Guinea-Bissau,GNB,1984,404,0,0,0.0,0.0,571.7829,636.3018,...,0.018248,-0.371771,-10.139670,8.471326,-9.362073,521.2217,-8,-7,-7.0,-7
4,Guinea-Bissau,GNB,1985,404,0,0,0.0,0.0,505.2217,571.7829,...,-0.035366,0.018248,-11.640992,-10.139670,8.471326,521.2217,-8,-8,-7.0,-7


In [20]:
icc_columns = [col for col in df_clim_conflict.columns if col.startswith("iccode")]
print(icc_columns)

['iccode1', 'iccode2', 'iccode3', 'iccode4', 'iccode5', 'iccode6', 'iccode7', 'iccode8', 'iccode9', 'iccode10', 'iccode11', 'iccode12', 'iccode13', 'iccode14', 'iccode15', 'iccode16', 'iccode17', 'iccode18', 'iccode19', 'iccode20', 'iccode21', 'iccode22', 'iccode23', 'iccode24', 'iccode25', 'iccode26', 'iccode27', 'iccode28', 'iccode29', 'iccode30', 'iccode31', 'iccode32', 'iccode33', 'iccode34', 'iccode35', 'iccode36', 'iccode37', 'iccode38', 'iccode39', 'iccode40', 'iccode41']


In [21]:
icc_year_columns = [col for col in df_clim_conflict.columns if col.startswith("Iccyear")]
print(icc_year_columns)

['Iccyear1', 'Iccyear2', 'Iccyear3', 'Iccyear4', 'Iccyear5', 'Iccyear6', 'Iccyear7', 'Iccyear8', 'Iccyear9', 'Iccyear10', 'Iccyear11', 'Iccyear12', 'Iccyear13', 'Iccyear14', 'Iccyear15', 'Iccyear16', 'Iccyear17', 'Iccyear18', 'Iccyear19', 'Iccyear20', 'Iccyear21', 'Iccyear22', 'Iccyear23', 'Iccyear24', 'Iccyear25', 'Iccyear26', 'Iccyear27', 'Iccyear28', 'Iccyear29', 'Iccyear30', 'Iccyear31', 'Iccyear32', 'Iccyear33', 'Iccyear34', 'Iccyear35', 'Iccyear36', 'Iccyear37', 'Iccyear38', 'Iccyear39', 'Iccyear40', 'Iccyear41']


In [22]:
#Sort Dataframe 
climate_conflict_table1 = df_clim_conflict[
    ["country"]+["year_actual"]+["war_prio_new"]+["temp_all"]+["temp_all_lag"]+
    ["prec_all"]+["prec_all_lag"]+["gdp_l"]+["polity2_lag"]+["year"]+icc_columns+icc_year_columns
]

In [23]:
#Define lists for the fixed effects variables in the table 1 regressions 
icc_columns_mo = [col for col in climate_conflict_table1.columns if col.startswith("iccode")]
icc_year_columns_mo = [col for col in climate_conflict_table1.columns if col.startswith("Iccyear")]

In [26]:
X1 = climate_conflict_table1[["temp_all"]+["temp_all_lag"]+icc_columns_mo+icc_year_columns_mo + ["war_prio_new"]].dropna()
y = X1["war_prio_new"]
X1 = climate_conflict_table1[["temp_all"]+["temp_all_lag"]+icc_columns_mo+icc_year_columns_mo].dropna()
#Add constant 
X1 = sm.add_constant(X1)
#Create model 1 regression
model1 = sm.OLS(y,X1).fit()
#Summary
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           war_prio_new   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.622
Method:                 Least Squares   F-statistic:                     18.59
Date:                Mon, 28 Apr 2025   Prob (F-statistic):          1.43e-137
Time:                        14:44:56   Log-Likelihood:                 246.53
No. Observations:                 889   AIC:                            -325.1
Df Residuals:                     805   BIC:                             77.32
Df Model:                          83                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.1099      0.730     -1.520      0.129      -2.544       0.324
temp_all         0.0447      0.022      2.077      0.038       0.002       0.087
temp_all_lag     0.0087      0.022      0.399      0.690      -0.034       0.052
iccode1         -0.3584      0.130     -2.753      0.006      -0.614      -0.103
iccode2         -0.3396      0.145     -2.347      0.019      -0.624      -0.056
iccode3         -0.4016      0.174     -2.313      0.021      -0.742      -0.061
iccode4         -0.3813      0.164     -2.331      0.020      -0.702      -0.060
iccode5         -0.3829      0.164     -2.337      0.020      -0.704      -0.061
iccode6         -0.3667      0.157     -2.341      0.019      -0.674      -0.059
iccode7         -0.3430      0.146     -2.343      0.019      -0.630      -0.056
iccode8         -0.2962      0.127     -2.338      0.020      -0.545      -0.048
iccode9         -0.2510      0.112     -2.246      0.025      -0.470      -0.032
iccode10        -0.4041      0.175     -2.305      0.021      -0.748      -0.060
iccode11        -0.1286      0.107     -1.202      0.230      -0.339       0.081
iccode12        -0.3933      0.118     -3.336      0.001      -0.625      -0.162
iccode13        -0.3604      0.154     -2.347      0.019      -0.662      -0.059
iccode14        -0.3474      0.148     -2.352      0.019      -0.637      -0.057
iccode15        -0.1957      0.099     -1.977      0.048      -0.390      -0.001
iccode16        -0.3311      0.141     -2.352      0.019      -0.607      -0.055
iccode17        -0.2126      0.102     -2.088      0.037      -0.412      -0.013
iccode18        -0.2102      0.102     -2.067      0.039      -0.410      -0.011
iccode19        -0.1325      0.134     -0.987      0.324      -0.396       0.131
iccode20        -0.2819      0.098     -2.869      0.004      -0.475      -0.089
iccode21        -0.4201      0.097     -4.348      0.000      -0.610      -0.230
iccode22         1.2460      0.099     12.604      0.000       1.052       1.440
iccode23        -0.2021      0.100     -2.014      0.044      -0.399      -0.005
iccode24        -0.0859      0.103     -0.833      0.405      -0.288       0.116
iccode25        -0.3332      0.137     -2.427      0.015      -0.603      -0.064
iccode26         0.0236      0.167      0.141      0.888      -0.305       0.352
iccode27        -0.0618      0.137     -0.451      0.652      -0.331       0.207
iccode28        -0.3584      0.154     -2.323      0.020      -0.661      -0.056
iccode29         1.2104      0.103     11.752      0.000       1.008       1.413
iccode30         1.1312      0.125      9.055      0.000       0.886       1.376
iccode31         1.3013      0.096     13.594      0.000       1.113       1.489
iccode32        -0.0510      0.112     -0.453      0.650      -0.272       0.170
iccod

In [27]:
#model 2
X2 = climate_conflict_table1[["temp_all"]+["temp_all_lag"]+["prec_all"]+["prec_all_lag"]+icc_columns_mo+icc_year_columns_mo
+ ["war_prio_new"]].dropna()
y2 = X2["war_prio_new"]
#Add constant 
X2 = climate_conflict_table1[["temp_all"]+["temp_all_lag"]+["prec_all"]+["prec_all_lag"]+icc_columns_mo+icc_year_columns_mo].dropna()
X2 = sm.add_constant(X2)
#Create model 2 regression
model2 = sm.OLS(y2,X2).fit()
#Summary
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           war_prio_new   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.621
Method:                 Least Squares   F-statistic:                     18.12
Date:                Mon, 28 Apr 2025   Prob (F-statistic):          2.30e-136
Time:                        14:47:07   Log-Likelihood:                 246.70
No. Observations:                 889   AIC:                            -321.4
Df Residuals:                     803   BIC:                             90.55
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.1771      0.787     -1.496      0.135      -2.722       0.367
temp_all         0.0430      0.023      1.900      0.058      -0.001       0.087
temp_all_lag     0.0132      0.023      0.566      0.572      -0.033       0.059
prec_all        -0.0230      0.059     -0.392      0.695      -0.138       0.092
prec_all_lag     0.0250      0.059      0.421      0.674      -0.092       0.141
iccode1         -0.3631      0.143     -2.538      0.011      -0.644      -0.082
iccode2         -0.3472      0.147     -2.368      0.018      -0.635      -0.059
iccode3         -0.4136      0.176     -2.353      0.019      -0.758      -0.069
iccode4         -0.3914      0.165     -2.372      0.018      -0.715      -0.068
iccode5         -0.3950      0.170     -2.331      0.020      -0.728      -0.062
iccode6         -0.3762      0.162     -2.328      0.020      -0.693      -0.059
iccode7         -0.3516      0.151     -2.326      0.020      -0.648      -0.055
iccode8         -0.3047      0.136     -2.244      0.025      -0.571      -0.038
iccode9         -0.2568      0.129     -1.989      0.047      -0.510      -0.003
iccode10        -0.4168      0.178     -2.342      0.019      -0.766      -0.068
iccode11        -0.1360      0.156     -0.873      0.383      -0.442       0.170
iccode12        -0.4034      0.180     -2.244      0.025      -0.756      -0.051
iccode13        -0.3720      0.162     -2.297      0.022      -0.690      -0.054
iccode14        -0.3583      0.155     -2.318      0.021      -0.662      -0.055
iccode15        -0.1998      0.113     -1.767      0.078      -0.422       0.022
iccode16        -0.3413      0.147     -2.329      0.020      -0.629      -0.054
iccode17        -0.2182      0.127     -1.718      0.086      -0.468       0.031
iccode18        -0.2143      0.110     -1.953      0.051      -0.430       0.001
iccode19        -0.1403      0.137     -1.021      0.307      -0.410       0.129
iccode20        -0.2863      0.116     -2.473      0.014      -0.513      -0.059
iccode21        -0.4235      0.109     -3.897      0.000      -0.637      -0.210
iccode22         1.2479      0.101     12.372      0.000       1.050       1.446
iccode23        -0.2041      0.101     -2.012      0.045      -0.403      -0.005
iccode24        -0.0836      0.103     -0.810      0.418      -0.286       0.119
iccode25        -0.3259      0.138     -2.361      0.018      -0.597      -0.055
iccode26         0.0346      0.169      0.204      0.838      -0.298       0.367
iccode27        -0.0693      0.140     -0.493      0.622      -0.345       0.206
iccode28        -0.3676      0.157     -2.343      0.019      -0.676      -0.060
iccode29         1.2139      0.106     11.471      0.000       1.006       1.422
iccode30         1.1372      0.127      8.940      0.000       0.888       1.387
iccod

In [28]:
X3 = climate_conflict_table1[["temp_all"]+["temp_all_lag"]+["prec_all"]+["prec_all_lag"]+["gdp_l"]+["polity2_lag"]+["year"]+
icc_columns_mo + ["war_prio_new"]].dropna()
y3 = X3["war_prio_new"]
X3 = climate_conflict_table1[["temp_all"]+["temp_all_lag"]+["prec_all"]+["prec_all_lag"]+["gdp_l"]+["polity2_lag"]+["year"]+
icc_columns_mo].dropna()
#Add constant 
X3 = sm.add_constant(X3)
#Create model 3 regression
model3 = sm.OLS(y3,X3).fit()
#Summary
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           war_prio_new   R-squared:                       0.389
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     11.12
Date:                Mon, 28 Apr 2025   Prob (F-statistic):           9.81e-57
Time:                        14:48:50   Log-Likelihood:                 27.536
No. Observations:                 815   AIC:                             34.93
Df Residuals:                     770   BIC:                             246.6
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.5215      0.983     -1.547      0.122      -3.452       0.409
temp_all         0.0489      0.029      1.698      0.090      -0.008       0.106
temp_all_lag     0.0206      0.030      0.694      0.488      -0.038       0.079
prec_all         0.0165      0.075      0.221      0.825      -0.130       0.163
prec_all_lag     0.0278      0.074      0.377      0.706      -0.117       0.173
gdp_l        -2.663e-05   2.11e-05     -1.260      0.208   -6.81e-05    1.49e-05
polity2_lag     -0.0005      0.002     -0.234      0.815      -0.005       0.004
year            -0.0041      0.002     -2.169      0.030      -0.008      -0.000
iccode1         -0.3077      0.150     -2.051      0.041      -0.602      -0.013
iccode2         -0.3575      0.152     -2.358      0.019      -0.655      -0.060
iccode3         -0.4088      0.189     -2.159      0.031      -0.780      -0.037
iccode4         -0.3766      0.172     -2.185      0.029      -0.715      -0.038
iccode5         -0.3663      0.155     -2.363      0.018      -0.671      -0.062
iccode6         -0.3508      0.172     -2.041      0.042      -0.688      -0.013
iccode7         -0.3170      0.151     -2.105      0.036      -0.613      -0.021
iccode8         -0.2611      0.119     -2.196      0.028      -0.494      -0.028
iccode9         -0.2285      0.113     -2.031      0.043      -0.449      -0.008
iccode10        -0.4153      0.179     -2.327      0.020      -0.766      -0.065
iccode11        -0.1043      0.164     -0.638      0.524      -0.425       0.217
iccode12        -0.2335      0.181     -1.294      0.196      -0.588       0.121
iccode13        -0.3536      0.150     -2.351      0.019      -0.649      -0.058
iccode14        -0.3334      0.135     -2.478      0.013      -0.598      -0.069
iccode15        -0.1493      0.090     -1.665      0.096      -0.325       0.027
iccode16        -0.3248      0.131     -2.478      0.013      -0.582      -0.067
iccode17        -0.0133      0.192     -0.069      0.945      -0.391       0.364
iccode18        -0.1954      0.077     -2.550      0.011      -0.346      -0.045
iccode19        -0.2068      0.132     -1.568      0.117      -0.466       0.052
iccode20        -0.0711      0.090     -0.789      0.430      -0.248       0.106
iccode21         0.0168      0.077      0.217      0.828      -0.135       0.169
iccode22         0.3942      0.069      5.737      0.000       0.259       0.529
iccode23        -0.1473      0.064     -2.299      0.022      -0.273      -0.022
iccode24        -0.0312      0.084     -0.373      0.709      -0.195       0.133
iccode25         0.2874      0.147      1.952      0.051      -0.002       0.576
iccode26         0.3859      0.195      1.977      0.048       0.003       0.769
iccode27        -0.1077      0.131     -0.822      0.411      -0.365       0.150
iccod

In [29]:
stargazer = Stargazer([model1, model2, model3])
stargazer.custom_columns(['Model 1', 'Model 2', 'Model 3'], [1, 1, 1])
stargazer.show_model_numbers(False)
stargazer.significant_digits(4)
stargazer.covariate_order(["temp_all", "temp_all_lag", "prec_all", "prec_all_lag", "gdp_l", "polity2_lag", "const"])
stargazer.title("Table 1. Regression coefficients on climate variables, with civil war as a dependent variable")

stargazer